#### ConversationEntityMemory is a memory class provided by LangChain, designed to track and store information about entities that arise in a conversation. It allows the AI to "remember" key facts about people, places, or concepts mentioned during a conversation, so it can refer back to them later on, improving the conversational experience.

## Key Features:
**Entity Tracking:** It identifies entities (e.g., names, places, concepts) and stores relevant information about them. For instance, if you mention "Tanmay" in one part of a conversation, it can remember details about "Tanmay" for later reference.

**Context-Aware:** It helps the AI maintain context by remembering details about the entities mentioned during the chat, ensuring more natural, fluid conversations over time.

**Customization:** You can customize what to store and how to retrieve it during future interactions.

# Conversation Entity Memory Demonstration

This notebook shows how to capture and recall entity-centric facts (people, places, concepts) across a conversation using LangChain's `ConversationEntityMemory`. It:

- Installs required dependencies
- Initializes a Gemini chat model (placeholder API key usage)
- Loads incremental inputs to extract entities & build summaries
- Uses `save_context` / `load_memory_variables` for evolving memory
- Builds a `ConversationChain` with an entity-aware prompt template
- Introspects the internal `entity_store`
- Demonstrates follow-up queries referencing previously mentioned entities

Each code cell is followed by an explanation block. All secrets are removed; replace any placeholders with your own secure credentials (e.g., environment variables, Colab secrets, or `.env`).

In [7]:
!pip install langchain

**Explanation:** Installs the core `langchain` framework providing memory abstractions, chains, and model interfaces.

In [8]:
!pip install -U langchain-community

**Explanation:** Upgrades `langchain-community` package containing integrations (loaders, tools, stores) separated from core LangChain.

In [9]:
!pip install langchain_google_genai

**Explanation:** Installs the Google Generative AI integration used to access Gemini chat models through LangChain.

In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI

**Explanation:** Imports the `ChatGoogleGenerativeAI` wrapper which standardizes Gemini chat responses for LangChain chains and memory modules.

In [11]:
import warnings
warnings.filterwarnings('ignore')

**Explanation:** Suppresses non-critical warnings for a cleaner educational output view.

In [ ]:
import os
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') or "Write your own password"  # Provide your Gemini API key securely
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

**Explanation:** Securely retrieves Gemini API key via Colab secrets with a placeholder fallback for open-source distribution. Replace placeholder with a real key locally; never commit secrets.

In [15]:
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite-001",convert_system_message_to_human=True)

**Explanation:** Instantiates the Gemini chat model; `convert_system_message_to_human=True` adapts system role prompts for this provider.

In [16]:
print(model.invoke("hi").content)

Hi there! How can I help you today?


In [17]:
from langchain.memory import ConversationEntityMemory

In [18]:
memory = ConversationEntityMemory(llm=model)

/tmp/ipython-input-1182818481.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationEntityMemory(llm=model)
/usr/local/lib/python3.11/dist-packages/pydantic/main.py:253: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [19]:
_input= {"input": "i am very hungry."}
memory.load_memory_variables(_input)

{'history': '', 'entities': {}}

In [20]:
_input= {"input": "Atif is working working on a hackathon project"}
memory.load_memory_variables(_input)

{'history': '', 'entities': {'Atif': ''}}

In [21]:
_input= {"input": "My name is asim, and I'm planning a trip to Paris."}
memory.load_memory_variables(_input)

{'history': '', 'entities': {'Asim': '', 'Paris': ''}}

In [22]:
_input= {"input": "atif is a great person who values gratitude."}
memory.load_memory_variables(_input)

{'history': '', 'entities': {'atif': ''}}

In [23]:
memory.save_context(
    {"Human": "Atif is working on a hackathon project"},
    {"AI": "That's awesome! What's the hackathon project about?"}
)

In [24]:
memory.load_memory_variables({"input": "who is atif?"})

{'history': "Human: Atif is working on a hackathon project\nAI: That's awesome! What's the hackathon project about?",
 'entities': {'Atif': ''}}

In [25]:
memory.save_context(
    {"Human": "It's a machine learning project focused on healthcare."},
     {"AI": "Sounds exciting! Are they building a prediction model or something else?"}
)

In [26]:
memory.save_context(
    {"Human": "Yes, they are building prediction model."},
    {"AI": "Wishing Sunny and Mayank all the best for their project!"}
)

In [27]:
memory.load_memory_variables({"input": "who is atif?"})

{'history': "Human: Atif is working on a hackathon project\nAI: That's awesome! What's the hackathon project about?\nHuman: It's a machine learning project focused on healthcare.\nAI: Sounds exciting! Are they building a prediction model or something else?\nHuman: Yes, they are building prediction model.\nAI: Wishing Sunny and Mayank all the best for their project!",
 'entities': {'Atif': 'Atif is building a prediction model.'}}

In [28]:
print('You are an AI assistant helping a human keep track of facts about relevant people, places, and concepts in their life. Update the summary of the provided entity in the "Entity" section based on the last line of your conversation with the human. If you are writing the summary for the first time, return a single sentence.\nThe update should only include facts that are relayed in the last line of conversation about the provided entity, and should only contain facts about the provided entity.\n\nIf there is no new information about the provided entity or the information is not worth noting (not an important or relevant fact to remember long-term), return the existing summary unchanged.\n\nFull conversation history (for context):\n{history}\n\nEntity to summarize:\n{entity}\n\nExisting summary of {entity}:\n{summary}\n\nLast line of conversation:\nHuman: {input}\nUpdated summary:')

You are an AI assistant helping a human keep track of facts about relevant people, places, and concepts in their life. Update the summary of the provided entity in the "Entity" section based on the last line of your conversation with the human. If you are writing the summary for the first time, return a single sentence.
The update should only include facts that are relayed in the last line of conversation about the provided entity, and should only contain facts about the provided entity.

If there is no new information about the provided entity or the information is not worth noting (not an important or relevant fact to remember long-term), return the existing summary unchanged.

Full conversation history (for context):
{history}

Entity to summarize:
{entity}

Existing summary of {entity}:
{summary}

Last line of conversation:
Human: {input}
Updated summary:


In [29]:
print('You are an AI assistant reading the transcript of a conversation between an AI and a human. Extract all of the proper nouns from the last line of conversation. As a guideline, a proper noun is generally capitalized. You should definitely extract all names and places.\n\nThe conversation history is provided just in case of a coreference (e.g. "What do you know about him" where "him" is defined in a previous line) -- ignore items mentioned there that are not in the last line.\n\nReturn the output as a single comma-separated list, or NONE if there is nothing of note to return (e.g. the user is just issuing a greeting or having a simple conversation).\n\nEXAMPLE\nConversation history:\nPerson #1: how\'s it going today?\nAI: "It\'s going great! How about you?"\nPerson #1: good! busy working on Langchain. lots to do.\nAI: "That sounds like a lot of work! What kind of things are you doing to make Langchain better?"\nLast line:\nPerson #1: i\'m trying to improve Langchain\'s interfaces, the UX, its integrations with various products the user might want ... a lot of stuff.\nOutput: Langchain\nEND OF EXAMPLE\n\nEXAMPLE\nConversation history:\nPerson #1: how\'s it going today?\nAI: "It\'s going great! How about you?"\nPerson #1: good! busy working on Langchain. lots to do.\nAI: "That sounds like a lot of work! What kind of things are you doing to make Langchain better?"\nLast line:\nPerson #1: i\'m trying to improve Langchain\'s interfaces, the UX, its integrations with various products the user might want ... a lot of stuff. I\'m working with Person #2.\nOutput: Langchain, Person #2\nEND OF EXAMPLE\n\nConversation history (for reference only):\n{history}\nLast line of conversation (for extraction):\nHuman: {input}\n\nOutput:')

You are an AI assistant reading the transcript of a conversation between an AI and a human. Extract all of the proper nouns from the last line of conversation. As a guideline, a proper noun is generally capitalized. You should definitely extract all names and places.

The conversation history is provided just in case of a coreference (e.g. "What do you know about him" where "him" is defined in a previous line) -- ignore items mentioned there that are not in the last line.

Return the output as a single comma-separated list, or NONE if there is nothing of note to return (e.g. the user is just issuing a greeting or having a simple conversation).

EXAMPLE
Conversation history:
Person #1: how's it going today?
AI: "It's going great! How about you?"
Person #1: good! busy working on Langchain. lots to do.
AI: "That sounds like a lot of work! What kind of things are you doing to make Langchain better?"
Last line:
Person #1: i'm trying to improve Langchain's interfaces, the UX, its integration

In [30]:
from langchain.chains import ConversationChain

In [31]:
from langchain.memory import ConversationEntityMemory

In [32]:
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

In [33]:
from pydantic import BaseModel

In [35]:
from typing import List, Dict, Any

In [36]:
conversation = ConversationChain(
    llm=model,
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=model)
)

/tmp/ipython-input-2685706177.py:1: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(


In [37]:
conversation.predict(input="Deven & Sam are working on a hackathon project")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"Okay, that's great! I'm ready to hear more about their hackathon project. What are they working on? What's the project about? I'm interested to know!"

In [38]:
conversation.memory.entity_store.store

{'Deven': 'Deven is working on a hackathon project.',
 'Sam': 'Sam is working on a hackathon project.'}

In [39]:
conversation.predict(input="They are trying to add more complex memory structures to Langchain")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"That's an interesting challenge! Adding more complex memory structures to Langchain could significantly improve its capabilities. I'm curious to know what kind of memory structures they're exploring. Are they looking at different types of memory, like episodic or semantic memory? Or are they focusing on improving the existing memory components? I'm also wondering what specific problems they are trying to solve by enhancing Langchain's memory."

In [40]:
conversation.predict(input="They are adding in a key-value store for entities mentioned so far in the conversation.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'That sounds like a very practical and useful addition! A key-value store for entities would allow Langchain to better track and utilize information about the entities discussed in the conversation. This could be used for a variety of things, such as:\n\n*   **Entity resolution:** Identifying and linking mentions of the same entity across different parts of the conversation.\n*   **Fact retrieval:** Quickly accessing information about specific entities.\n*   **Contextual understanding:** Remembering and using information about entities to understand the nuances of the conversation.\n\nWhat key-value store are they using? How are they planning to populate and update the store?'

In [41]:
conversation.predict(input="What do you know about Deven & Sam?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'I know that both Deven and Sam are working on a hackathon project together. They are currently focused on enhancing Langchain by adding a key-value store to track entities mentioned in the conversation.'

In [42]:
from pprint import pprint
pprint(conversation.memory.entity_store.store)

{'Deven': 'Deven is working on a hackathon project.',
 'Langchain': 'They are trying to add more complex memory structures to it.',
 'Sam': 'Sam is working on a hackathon project.'}


In [43]:
conversation.predict(input="Sam is the founder of a company called Daimon.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"That's interesting! So Sam is the founder of a company called Daimon, and he's working on a hackathon project with Deven that involves improving Langchain. This suggests that the hackathon project might be related to Daimon's work or interests. It would be interesting to know more about Daimon and how Sam's work there relates to the Langchain project."

In [44]:
from pprint import pprint
pprint(conversation.memory.entity_store.store)

{'Daimon': 'Sam is the founder of a company called Daimon.',
 'Deven': 'Deven is working on a hackathon project.',
 'Langchain': 'They are trying to add more complex memory structures to it.',
 'Sam': 'Sam is the founder of a company called Daimon.'}


In [45]:
conversation.predict(input="What do you know about Sam?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'I know that Sam is the founder of a company called Daimon. He is also working on a hackathon project with Deven to improve Langchain by adding a key-value store for entities mentioned in the conversation.'

# About the Author

<div style="background-color: #f8f9fa; border-left: 5px solid #28a745; padding: 20px; margin-bottom: 20px; border-radius: 5px;">
  <h2 style="color: #28a745; margin-top: 0; font-family: 'Poppins', sans-serif;">Muhammad Atif Latif</h2>
  <p style="font-size: 16px; color: #495057;">Data Scientist & Machine Learning Engineer</p>
  
  <p style="font-size: 15px; color: #6c757d; margin-top: 15px;">
    Passionate about building AI solutions that solve real-world problems. Specialized in machine learning,
    deep learning, and data analytics with experience implementing production-ready models.
  </p>
</div>

## Connect With Me

<div style="display: flex; flex-wrap: wrap; gap: 10px; margin-top: 15px;">
  <a href="https://github.com/m-Atif-Latif" target="_blank">
    <img src="https://img.shields.io/badge/GitHub-Follow-212121?style=for-the-badge&logo=github" alt="GitHub">
  </a>
  <a href="https://www.kaggle.com/matiflatif" target="_blank">
    <img src="https://img.shields.io/badge/Kaggle-Profile-20BEFF?style=for-the-badge&logo=kaggle" alt="Kaggle">
  </a>
  <a href="https://www.linkedin.com/in/muhammad-atif-latif-13a171318" target="_blank">
    <img src="https://img.shields.io/badge/LinkedIn-Connect-0077B5?style=for-the-badge&logo=linkedin" alt="LinkedIn">
  </a>
  <a href="https://x.com/mianatif5867" target="_blank">
    <img src="https://img.shields.io/badge/Twitter-Follow-1DA1F2?style=for-the-badge&logo=twitter" alt="Twitter">
  </a>
  <a href="https://www.instagram.com/its_atif_ai/" target="_blank">
    <img src="https://img.shields.io/badge/Instagram-Follow-E4405F?style=for-the-badge&logo=instagram" alt="Instagram">
  </a>
  <a href="mailto:muhammadatiflatif67@gmail.com">
    <img src="https://img.shields.io/badge/Email-Contact-D14836?style=for-the-badge&logo=gmail" alt="Email">
  </a>
</div>

---

## Notebook Summary

In this walkthrough you:
- Installed LangChain core + community + Gemini integration.
- Initialized a Gemini model with secure (placeholder) key loading.
- Created a `ConversationEntityMemory` instance to capture entities and their evolving summaries.
- Incrementally fed inputs containing names and contextual facts (Atif, Sam, Deven) and persisted them via `save_context`.
- Queried memory using `load_memory_variables` to retrieve relevant entity information for follow-up questions.
- Used the built-in `ENTITY_MEMORY_CONVERSATION_TEMPLATE` within a `ConversationChain` to automatically extract and update entity summaries.
- Inspected the underlying `entity_store` to verify stored structured knowledge.

Why it matters: Entity memory enables richer, more personalized conversational agents by grounding responses in previously mentioned facts without overloading token context.

Crafted and curated by **Muhammad Atif Latif** — delivering practical, production-focused AI. Star the repo and connect below to follow more LangChain, RAG, and MLOps explorations.